## Maximum Likelihood Estimation(MLE)

예를 들어, 이항 분포에서 $\theta$가 어떠한 확률 값이라고 할 때,  
Observation을 가장 잘 설명해주는 $\theta$ 값을 찾아내는 과정을 MLE라고 한다.

Gradient Descent를 이용하여 가장 최대가 되는 Likelihood가 되는 지점을 찾는다.

## Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility - Random Seed를 고정해주기 위해서
torch.manual_seed(1)

## Training and Test Datasets

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    def forward(self, x):
        return self.linear(x)

In [6]:
# 모델 생성
model = SoftmaxClassifierModel()

# Optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

print('--------------------')
print('Model Parameter List')
print('--------------------')
for name,params in model.named_parameters():
    print(name)
    print(params)

--------------------
Model Parameter List
--------------------
linear.weight
Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462],
        [-0.1188,  0.2937,  0.0803]], requires_grad=True)
linear.bias
Parameter containing:
tensor([-0.0707,  0.1601,  0.0285], requires_grad=True)


### Train function

In [7]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.cross_entropy(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

### Test funtion

In [8]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

### Execute train function

In [9]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


### Execute test function

In [10]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


## Learning Rate

Gradient Descent 에서의 $\alpha$ 값

`optimizer = optim.SGD(model.parameters(), lr=0.1)` 에서 `lr=0.1` 이다
'lr'을 이용하여 learning rate 조절

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [11]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [12]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 0.983424
Epoch    1/20 Cost: 3236.111816
Epoch    2/20 Cost: 1445972.500000
Epoch    3/20 Cost: 624019.125000
Epoch    4/20 Cost: 1415479.875000
Epoch    5/20 Cost: 1536597.500000
Epoch    6/20 Cost: 1466828.500000
Epoch    7/20 Cost: 722535.062500
Epoch    8/20 Cost: 1047452.812500
Epoch    9/20 Cost: 1296729.875000
Epoch   10/20 Cost: 760035.062500
Epoch   11/20 Cost: 2123078.750000
Epoch   12/20 Cost: 57877.050781
Epoch   13/20 Cost: 355265.343750
Epoch   14/20 Cost: 997428.562500
Epoch   15/20 Cost: 1085792.250000
Epoch   16/20 Cost: 1338703.750000
Epoch   17/20 Cost: 1478785.000000
Epoch   18/20 Cost: 902766.125000
Epoch   19/20 Cost: 788170.187500


learning rate이 너무 작으면 cost가 거의 줄어들지 않는다.

In [13]:
model = SoftmaxClassifierModel()

In [14]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [15]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 1.280268
Epoch    2/20 Cost: 1.280268
Epoch    3/20 Cost: 1.280268
Epoch    4/20 Cost: 1.280268
Epoch    5/20 Cost: 1.280268
Epoch    6/20 Cost: 1.280268
Epoch    7/20 Cost: 1.280268
Epoch    8/20 Cost: 1.280268
Epoch    9/20 Cost: 1.280268
Epoch   10/20 Cost: 1.280268
Epoch   11/20 Cost: 1.280268
Epoch   12/20 Cost: 1.280268
Epoch   13/20 Cost: 1.280268
Epoch   14/20 Cost: 1.280268
Epoch   15/20 Cost: 1.280268
Epoch   16/20 Cost: 1.280268
Epoch   17/20 Cost: 1.280268
Epoch   18/20 Cost: 1.280268
Epoch   19/20 Cost: 1.280268


적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [16]:
model = SoftmaxClassifierModel()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [18]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 1.334307
Epoch    2/20 Cost: 1.047911
Epoch    3/20 Cost: 0.996043
Epoch    4/20 Cost: 0.985740
Epoch    5/20 Cost: 0.977224
Epoch    6/20 Cost: 0.970065
Epoch    7/20 Cost: 0.963589
Epoch    8/20 Cost: 0.957561
Epoch    9/20 Cost: 0.951825
Epoch   10/20 Cost: 0.946302
Epoch   11/20 Cost: 0.940942
Epoch   12/20 Cost: 0.935719
Epoch   13/20 Cost: 0.930613
Epoch   14/20 Cost: 0.925613
Epoch   15/20 Cost: 0.920711
Epoch   16/20 Cost: 0.915902
Epoch   17/20 Cost: 0.911182
Epoch   18/20 Cost: 0.906547
Epoch   19/20 Cost: 0.901994


## Data Preprocessing (데이터 전처리)

데이터를 zero-center하고 normalize하자.  
값이 큰 데이터에 대해 편향되게 학습이 진행될 수 있기 때문이다.

In [19]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [20]:
mu = x_train.mean(dim=0)    # 첫번째 차원을 기준으로 평균을 구한다.

In [21]:
sigma = x_train.std(dim=0)  # 첫번째 차원을 기준으로 표준편차를 구한다.

In [22]:
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Normalize와 zero center한 X로 학습해서 성능을 보자

In [23]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [24]:
model = MultivariateLinearRegressionModel()

In [25]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [26]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [27]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29729.951172
Epoch    1/20 Cost: 18889.082031
Epoch    2/20 Cost: 12048.976562
Epoch    3/20 Cost: 7699.843750
Epoch    4/20 Cost: 4924.700195
Epoch    5/20 Cost: 3151.020264
Epoch    6/20 Cost: 2016.562988
Epoch    7/20 Cost: 1290.709106
Epoch    8/20 Cost: 826.216003
Epoch    9/20 Cost: 528.952271
Epoch   10/20 Cost: 338.703339
Epoch   11/20 Cost: 216.940048
Epoch   12/20 Cost: 139.006989
Epoch   13/20 Cost: 89.125130
Epoch   14/20 Cost: 57.196079
Epoch   15/20 Cost: 36.757317
Epoch   16/20 Cost: 23.672049
Epoch   17/20 Cost: 15.293400
Epoch   18/20 Cost: 9.927166
Epoch   19/20 Cost: 6.488903


## Overfitting

너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.  

이것을 방지하는 방법은 크게 세 가지인데:

1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. **Regularization**

Regularization: Let's not have too big numbers in the weights

In [28]:
def train_with_regularization(model, optimizer, x_train, y_train):
    epochs = 20
    for epoch in range(epochs):
        
        #H(x) 계산
        prediction = model(x_train)
        
        #Cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # L2 norm 계산 (loss function)
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
        
        cost += l2_reg
        
        #cost로 H(x) 계선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, epochs, cost.item()
        ))

In [29]:
# 모델 생성
model = MultivariateLinearRegressionModel()

# optimizer 생성
optimizer = optim.SGD(model.parameters(), lr=1e-1)

### Excute train

In [30]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 29632.869141
Epoch    2/20 Cost: 18897.533203
Epoch    3/20 Cost: 12122.802734
Epoch    4/20 Cost: 7814.004395
Epoch    5/20 Cost: 5064.236328
Epoch    6/20 Cost: 3306.669678
Epoch    7/20 Cost: 2182.488770
Epoch    8/20 Cost: 1463.201782
Epoch    9/20 Cost: 1002.911377
Epoch   10/20 Cost: 708.338318
Epoch   11/20 Cost: 519.812378
Epoch   12/20 Cost: 399.154022
Epoch   13/20 Cost: 321.930054
Epoch   14/20 Cost: 272.503662
Epoch   15/20 Cost: 240.868011
Epoch   16/20 Cost: 220.618408
Epoch   17/20 Cost: 207.656235
Epoch   18/20 Cost: 199.357986
Epoch   19/20 Cost: 194.044785
Epoch   20/20 Cost: 190.642242
